# `Channel` (synchronization of tasks)

Implement synchronization/communication between tasks.

Most relevant functions: `put!`, `take!`, `fetch`, `isready` and `wait`.

In [1]:
ch = Channel{Int}(5) # a channel that can hold up to 5 integers

Channel{Int64}(5) (empty)

In [2]:
isready(ch) # something in the channel?

false

In [3]:
put!(ch, 3)

3

In [4]:
isready(ch)

true

In [5]:
3+3

6

In [6]:
fetch(ch)

3

In [7]:
take!(ch)

3

In [8]:
isready(ch)

false

In [9]:
put!(ch, 4)

4

In [10]:
fetch(ch)

4

In [11]:
take!(ch)

4

**Be careful**, `take!` and `put!` are blocking if the channel is empty or full!

In [12]:
isready(ch)

false

In [13]:
# take!(ch) if we execute this, while isready(ch) == false, the current Julia session will hang.

## Basic example

In [14]:
c = Channel{Union{Nothing, Int}}(1)

background_task = Threads.@spawn begin
    while isopen(c)
        el = take!(c)
        if !isnothing(el)
            println("Took element $el from channel.")
        else
            println("Closing channel (saw `nothing`).")
            close(c)
        end
    end
end

Task (runnable) @0x00001490c5c233a0

In [15]:
put!(c, 3);

Took element 3 from channel.


In [16]:
put!(c, nothing);

Closing channel (saw `nothing`).


In [17]:
background_task

Task (done) @0x00001490c5c233a0

Multiple (two) tasks:

In [18]:
c = Channel{Union{Nothing, Int}}(1)

background_task = Threads.@spawn begin
    while isopen(c)
        el = take!(c)
        if !isnothing(el)
            println("Task1: Took element $el from channel.")
        else
            println("Task1: Closing channel (saw nothing).")
            close(c)
        end
    end
end

putter_task = Threads.@spawn begin
    for i in 1:10
        println("Task2: Putting $i into channel")
        put!(c, i)
    end
    println("Task2: Putting nothing into channel")
    put!(c, nothing)
end;

Task2: Putting 1 into channel
Task2: Putting 2 into channel
Task2: Putting 3 into channel
Task1: Took element 1 from channel.
Task1: Took element 2 from channel.
Task1: Took element 3 from channel.
Task2: Putting 4 into channel
Task2: Putting 5 into channel
Task2: Putting 6 into channel
Task1: Took element 4 from channel.
Task1: Took element 5 from channel.
Task1: Took element 6 from channel.
Task2: Putting 7 into channel
Task2: Putting 8 into channel
Task2: Putting 9 into channel
Task1: Took element 7 from channel.
Task1: Took element 8 from channel.
Task1: Took element 9 from channel.
Task2: Putting 10 into channel
Task2: Putting nothing into channel
Task1: Took element 10 from channel.
Task1: Closing channel (saw nothing).


## Example: parallel `findany`

`findany(pred, data)`: find **any** element in `data` for which the predicate (`pred`) is true. Return `nothing` if no element meets `pred`.

(Compare to e.g. `findfirst(pred, data)`.)

In [19]:
using ChunkSplitters

function safe_put!(c, i)
    try
        put!(c, i)
    catch e
        # handle the case that the channel could have been closed in the meantime
        e isa InvalidStateException || rethrow()
    end
end

function findany(pred, data; nchunks=Threads.nthreads())
    # Channel to synchronize tasks. Will either hold a result (Int) or nothing.
    c = Channel{Union{Nothing,Int}}(1)

    # One task per chunk
    tasks = map(chunks(data, nchunks)) do (chunk_idcs, ichunk)
        Threads.@spawn begin
            found = false
            for i in chunk_idcs
                # if the channel is closed, some other task has already found something → stop
                if !isopen(c)
                    break
                end

                if pred(data[i])
                    # found element, try to put it into the channel
                    safe_put!(c, i)
                    found = true
                    break
                end
            end
            if !found
                # the task hasn't found an element in its chunk → put `nothing` into the channel
                safe_put!(c, nothing)
            end
        end
    end

    # wait until we either have a result or all tasks have finished
    cn = 0
    result = nothing
    while true
        num = take!(c)
        cn += 1
        if num isa Int
            result = num
            break
        elseif cn == nchunks
            break
        end
    end
    # close channel to notify all other tasks that they should stop
    close(c)

    # wait for all tasks to terminate
    foreach(wait, tasks)

    return result
end

findany (generic function with 1 method)

In [20]:
test = rand(1:10, 1000);

function pred(x) # "expensive" predicate
    sleep(0.1)
    x == 10
end

findany(pred, test)

252

In [21]:
using BenchmarkTools

@btime findfirst($pred, $test) samples = 5 evals = 3;
@btime findany($pred, $test) samples = 5 evals = 3;

  1.719 s (104 allocations: 2.99 KiB)
  302.897 ms (136 allocations: 7.58 KiB)
